In [1]:
import os
import numpy as np
import pandas as pd
from copy import copy
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, Normalizer, RobustScaler, MinMaxScaler
from sklearn.model_selection import (cross_val_score, cross_val_predict, cross_validate,
                                     StratifiedKFold, KFold, GroupKFold, train_test_split)
from bokeh.models.widgets import TextInput, Panel, Tabs
from bokeh.models import CustomJS, ColumnDataSource, Slider, Div
from bokeh import events
from bokeh.plotting import figure, show, curdoc
from bokeh.io import output_file
from bokeh.layouts import gridplot, layout, column, row, widgetbox
from bokeh.models import HoverTool, CrosshairTool, Div
from sklearn.neural_network._base import ACTIVATIONS

from surface3d import Surface3d

np.set_printoptions(precision=3)

In [2]:
from bokeh.models import tickers

In [3]:
SAVE_MODEL = False
LOAD_MODEL = False

In [4]:
features = ['avg_warhead_load',
            'weighted_avg_yld',
            'var_warhead_load',
            'probability_penetrate',
            'boost_reliability',
            'soa',
            'hob',
            'launch_reliability',
            'rb_reliability',
            'cep']
fnames = ['Average Warhead Load',
          'Weighted Average Yield',
          'Variance Warhead Load',
          'Probability Penetrate',
          'Boost Reliability',
          'Speed of Advance',
          'Height of Burst',
          'Launch Reliability',
          'Release Body Reliability',
          'Circular Error Probable']
vnames = ['avg_wh', 'yld', 'var_wh', 'pene', 'boost', 'soa', 'hob', 'launch', 'rb_r', 'cep']
response = ['missiles_expended', 'total_wh_expended']


In [5]:
if LOAD_MODEL:
    with open('C:\\Users\\jmccrary\\moving_surface\\model_coefficients.json', 'r') as f:
        coefsource = ColumnDataSource(json.load(f)['data'])
    with open("C:\\Users\\jmccrary\\moving_surface\\feature_scales.json", 'r') as f:
        psource = ColumnDataSource(json.load(f)['data'])
        
    scalers = [MinMaxScaler().fit(np.array(psource.data[v + '_bnds']).reshape((2,1))) for v in vnames]

else:
    df = pd.read_csv('..\\demo_output.csv', index_col=None)

    df.drop(df.index[df.total_cost.isnull()], inplace=True)
    df.index = np.arange(len(df))
    N = len(df)
    scalers = [MinMaxScaler().fit(df[f].values.reshape(N, 1)) for f in features]
    
    fscld = [s.transform(df[f].values.reshape(N, 1)) for s, f in zip(scalers, features)]
    X = np.hstack(fscld)
    y1 = df[response[0]].values
    y2 = df[response[1]].values
    
    respo1 = MLPRegressor(hidden_layer_sizes=(15,), activation='tanh', 
                      max_iter=20000, tol=1e-9)
    
    respo2 = MLPRegressor(hidden_layer_sizes=(15,), activation='tanh', 
                      max_iter=20000, tol=1e-9)
    respo1.fit(X, y1)
    print('Fit response surface 1 with', respo1.n_iter_, 'iterations.')

    respo2.fit(X, y2)
    print('Fit response surface 2 with', respo2.n_iter_, 'iterations.')
    
    coefsource = ColumnDataSource(data=dict(coef=[[c.tolist() for c in respo1.coefs_], [c.tolist() for c in respo2.coefs_]],
                                        cept=[[c.tolist() for c in respo1.intercepts_], [c.tolist() for c in respo2.intercepts_]]))
    
    
    N = len(df)
    Xreg = np.hstack([df[f].values.reshape(N, 1) for f in features])
    N = 64
    xmeans = Xreg.mean(axis=0)
    xpts = np.array([sc.transform(x)[0,0] for x, sc in zip(xmeans, scalers)]).reshape(1, 10)
    mmpts = xpts * np.ones((N, 1))
    xs = []
    y1s = []
    y2s = []
    x_ = np.linspace(0, 1, N)
    for i, (f, sc) in enumerate(zip(features, scalers)):
        xs.append(np.linspace(df[f].min(), df[f].max(), N))
        xx = copy(mmpts)
        xx[:, i] = x_
        y1_f = respo1.predict(xx)
        y2_f = respo2.predict(xx)
        y1s.append(y1_f)
        y2s.append(y2_f)

    shortdata = {}
    for key, ray in dict(y1=y1s, y2=y2s, x=xs).items():
        for i in range(len(ray)):
            shortdata[vnames[i] + '_' + key] = ray[i]
    shortdata['x_'] = np.linspace(0, 1, N)
    shortdata['zeros'] = x_ * 0
    
    # set up Point Source which has data bounds and current selected point
    pdict = {}
    r1val = respo1.predict(np.vstack([xpts]*2))
    r2val = respo2.predict(np.vstack([xpts]*2))
    pdict['r1val'] = r1val.tolist()
    pdict['r2val'] = r2val.tolist()

    psource = ColumnDataSource(pdict)
    
def r1_predict(X):
    c0 = np.array(coefsource.data['coef'][0][0])
    c1 = np.array(coefsource.data['coef'][0][1])
    b0 = np.array(coefsource.data['cept'][0][0])
    b1 = np.array(coefsource.data['cept'][0][1])
    return (np.tanh(X.dot(c0) + b0).dot(c1) + b1)[:, 0]

def r2_predict(X):
    c0 = np.array(coefsource.data['coef'][1][0])
    c1 = np.array(coefsource.data['coef'][1][1])
    b0 = np.array(coefsource.data['cept'][1][0])
    b1 = np.array(coefsource.data['cept'][1][1])
    return (np.tanh(X.dot(c0) + b0).dot(c1) + b1)[:, 0]

Fit response surface 1 with 141 iterations.
Fit response surface 2 with 195 iterations.


In [69]:
if SAVE_MODEL:
    with open('C:\\Users\\jmccrary\\moving_surface\\model_coefficients.json', 'w') as f:
        f.write(coefsource.to_json_string(False))
    with open("C:\\Users\\jmccrary\\moving_surface\\feature_scales.json", 'w') as f:
        f.write(psource.to_json_string(False))
    

In [6]:
xpts

array([[ 0.248,  0.294,  0.163,  0.501,  0.504,  0.499,  0.501,  0.495,
         0.502,  0.501]])

In [7]:
# load tcode, callback function for the text inputs, from file
with open('C:\\users\\jmccrary\\moving_surface\\moving_profiles_and_surface.js', 'r') as f:
    textcode = f.read()

In [11]:
df.columns

Index(['boost_reliability', 'cep', 'hob', 'launch_reliability', 'missile_type',
       'missiles_expended', 'multi_missile', 'probability_penetrate',
       'rb_reliability', 'single_missile', 'soa', 'avg_warhead_load',
       'var_warhead_load', 'weighted_avg_yld', 'total_wh_expended',
       'total_cost', 'coverage_norm', 'msn_success_norm',
       'Predicted missiles_expended', 'Predicted total_wh_expended',
       'Total_utility', 'Pareto_Utility', 'Predicted Total_utility',
       'Predicted total_cost', 'Predicted coverage_norm',
       'Predicted msn_success_norm'],
      dtype='object')

In [8]:
output_file('profiler_layout.html', title='profiler')

hover = HoverTool(tooltips=[("avg_wh", "$x"), ("r1", "@avg_wh_y1")], mode='vline', names=["r_surf"])

pargs = {'plot_width': 130, 'plot_height': 100, 'tools':'wheel_zoom', 'toolbar_location':None}

outdiv = Div(width=1324)

style = {'font-size': '1.25em',
         'text-align': 'center',
         'vertical-align': 'middle',
         'margin-top': 0}

inputstyle = {'margin-left': 10,
              'margin-right': 10}

source = ColumnDataSource(data=shortdata)
psource = ColumnDataSource(data=pdict)

t0_ps = [Div(text=fnames[0], width=145, height=40, style=style)]
val = pdict[vnames[0] + '_p'][0] * (pdict[vnames[0] + '_bnds'][1] - pdict[vnames[0] + '_bnds'][0]) + pdict[vnames[0] + '_bnds'][0]
textrow = [TextInput(value='{:0.3f}'.format(val), title=None, width=145)]
callback = CustomJS(args=dict(psource=psource, source=source, coefsource=coefsource, outdiv=outdiv),
                        code=textcode % "avg_wh")
textrow[0].js_on_change('value', callback)

for i in range(1, 10):
    t0_ps.append(Div(text=fnames[i], width=130, height=40, style=style))
    val = pdict[vnames[i] + '_p'][0] * (pdict[vnames[i] + '_bnds'][1] - pdict[vnames[i] + '_bnds'][0]) + pdict[vnames[i] + '_bnds'][0]
    textrow.append(TextInput(value='{:0.3f}'.format(val), title=None, width=130))
    callback = CustomJS(args=dict(psource=psource, source=source, coefsource=coefsource, outdiv=outdiv),
                        code=textcode % vnames[i])
    textrow[-1].js_on_change('value', callback)


r1val = respo1.predict(np.vstack([xpts]*2))

r1 = figure(plot_width=154, plot_height=100, tools='wheel_zoom', toolbar_location=None)
r1.line(x='avg_wh_x', y='avg_wh_y1', line_width=3, alpha=0.5, line_cap='round', name='r_surf', source=source)
r1.line('avg_wh_bnds', 'r1val', color='black', line_width=1, alpha=0.7, name='r_value', source=psource)
r1.min_border_right = 0

r1.toolbar.active_scroll = r1.tools[0]
#r1.toolbar.active_drag = r1.tools[1]
r1.add_tools(hover)
r1.xaxis.visible = False
r1.grid.grid_line_alpha = 0.4

callback = CustomJS(args=dict(myText=textrow[0]),
                    code="""myText.value = cb_obj['x'].toPrecision(5);""")
r1.js_on_event(events.Tap, callback)
r1.js_on_event(events.Pan, callback)

r1_ps = [r1]
for i in range(1, 10):
    vn = vnames[i]
    r1_ps.append(figure(y_range=r1.y_range, **pargs))
    r1_ps[-1].xaxis.visible = False
    r1_ps[-1].yaxis.visible = False
    r1_ps[-1].min_border = 0
    r1_ps[-1].line(x=vn + '_x', y=vn + '_y1', line_width=3, alpha=0.5, line_cap='round', name='r_surf', source=source)
    r1_ps[-1].line(vn+'_bnds', 'r1val', color='black', line_width=1, alpha=0.7, name='r_value', source=psource)
    #r1_ps[-1].toolbar.active_drag = r1_ps[-1].tools[??]
    r1_ps[-1].toolbar.active_scroll = r1_ps[-1].tools[0]
    hover = HoverTool(tooltips=[(vn, "$x"), ("r1", "@{}_y1".format(vn))], mode='vline', names=["r_surf"])
    r1_ps[-1].add_tools(hover)
    r1_ps[-1].grid.grid_line_alpha = 0.4
    callback = CustomJS(args=dict(myText=textrow[i]),
                        code="""myText.value = cb_obj['x'].toPrecision(5);""")
    r1_ps[-1].js_on_event(events.Tap, callback)
    r1_ps[-1].js_on_event(events.Pan, callback)
    
pargs = {'plot_width': 130, 'plot_height': 119, 'tools':'wheel_zoom', 'toolbar_location':None}

r2val = respo2.predict(np.vstack([xpts]*2))
        
r2 = figure(plot_width=154, plot_height=119, tools='wheel_zoom', 
            toolbar_location=None, x_range=r1.x_range)
r2.line(x='avg_wh_x', y='avg_wh_y2', line_width=3, alpha=0.5, line_cap='round', name='r_surf', source=source)
#r2.line(xs[0], tryy, line_width=2, alpha=0.5, color='firebrick', line_cap='round', name='testy')
r2.line('avg_wh_bnds', 'r2val', color='black', line_width=1, alpha=0.7, name='r_value', source=psource)
r2.min_border_right = 0
#r2.toolbar.active_drag = r2.tools[1]
r2.toolbar.active_scroll = r2.tools[0]
hover = HoverTool(tooltips=[('avg_wh', "$x"), ("r2", "@avg_wh_y2")], mode='vline', names=["r_surf"])
r2.add_tools(hover)
r2.grid.grid_line_alpha = 0.4
#r2.xaxis.axis_label = fnames[0]
#r2.xaxis.axis_label_text_font_size = '1em'
callback = CustomJS(args=dict(myText=textrow[0]),
                    code="""myText.value = cb_obj['x'].toPrecision(5);""")
r2.js_on_event(events.Tap, callback)
r2.js_on_event(events.Pan, callback)

r2_ps = [r2]
for i in range(1, 10):
    vn = vnames[i]
    r2_ps.append(figure(y_range=r2.y_range, x_range=r1_ps[i].x_range, **pargs))
    r2_ps[-1].yaxis.visible = False
    r2_ps[-1].min_border = 0
    r2_ps[-1].line(x=vn + '_x', y=vn + '_y2', line_width=3, alpha=0.5, line_cap='round', name='r_surf', source=source)
    r2_ps[-1].line(vn+'_bnds', 'r2val', color='black', line_width=1, alpha=0.7, name='r_value', source=psource)
    #r2_ps[-1].toolbar.active_drag = r2_ps[-1].tools[1]
    r2_ps[-1].toolbar.active_scroll = r2_ps[-1].tools[0]
    hover = HoverTool(tooltips=[(vn, "$x"), ("r2", "@{}_y2".format(vn))], mode='vline', names=['r_surf'])
    r2_ps[-1].add_tools(hover)
    r2_ps[-1].grid.grid_line_alpha = 0.4
    callback = CustomJS(args=dict(myText=textrow[i]),
                        code="""myText.value = cb_obj['x'].toPrecision(5);""")
    r2_ps[-1].js_on_event(events.Tap, callback)
    r2_ps[-1].js_on_event(events.Pan, callback)


show(column(row(t0_ps), row(textrow), row(r1_ps), row(r2_ps), outdiv))

KeyError: 'avg_wh_p'

In [10]:
pdict

{'r1val': [4.618988402285122, 4.618988402285122],
 'r2val': [21.224083382270436, 21.224083382270436]}

In [9]:
output_file('profiler_col_first_layout.html', title='profiler')

hover = HoverTool(tooltips=[("avg_wh", "$x"), ("r1", "@avg_wh_y1")], mode='vline', names=["r_surf"])

pargs = {'plot_width': 130, 'plot_height': 100, 'tools':'wheel_zoom', 'toolbar_location':None}

outdiv = Div(width=1324)

style = {'font-size': '1.25em',
         'text-align': 'center',
         'vertical-align': 'middle',
         'margin-top': 0}

inputstyle = {'margin-left': 10,
              'margin-right': 10}

source = ColumnDataSource(data=shortdata)
psource = ColumnDataSource(data=pdict)

t0_ps = []
textrow = []

for i in range(0, 10):
    t0_ps.append(Div(text=fnames[i], width=130, height=40, style=style))
    val = pdict[vnames[i] + '_p'][0] * (pdict[vnames[i] + '_bnds'][1] - pdict[vnames[i] + '_bnds'][0]) + pdict[vnames[i] + '_bnds'][0]
    textrow.append(TextInput(value='{:0.3f}'.format(val), title=None, css_classes=['myTextInputFix']))
    callback = CustomJS(args=dict(psource=psource, source=source, coefsource=coefsource, outdiv=outdiv),
                        code=textcode % vnames[i])
    textrow[-1].js_on_change('value', callback)


r1val = respo1.predict(np.vstack([xpts]*2))


r1_ps = []
for i in range(0, 10):
    vn = vnames[i]
    if i == 0:
        r1_ps.append(figure(**pargs))
    else:
        r1_ps.append(figure(y_range=r1_ps[0].y_range, **pargs))
    r1_ps[-1].xaxis.visible = False
    r1_ps[-1].yaxis.visible = False
    r1_ps[-1].min_border = 0
    r1_ps[-1].line(x=vn + '_x', y=vn + '_y1', line_width=3, alpha=0.5, line_cap='round', name='r_surf', source=source)
    r1_ps[-1].line(vn+'_bnds', 'r1val', color='black', line_width=1, alpha=0.7, name='r_value', source=psource)
    #r1_ps[-1].toolbar.active_drag = r1_ps[-1].tools[??]
    r1_ps[-1].toolbar.active_scroll = r1_ps[-1].tools[0]
    hover = HoverTool(tooltips=[(vn, "$x"), ("r1", "@{}_y1".format(vn))], mode='vline', names=["r_surf"])
    r1_ps[-1].add_tools(hover)
    r1_ps[-1].grid.grid_line_alpha = 0.4
    callback = CustomJS(args=dict(myText=textrow[i]),
                        code="""myText.value = cb_obj['x'].toPrecision(5);""")
    r1_ps[-1].js_on_event(events.Tap, callback)
    r1_ps[-1].js_on_event(events.Pan, callback)
    
#pargs = {'plot_width': 130, 'plot_height': 119, 'tools':'wheel_zoom', 'toolbar_location':None}

r2val = respo2.predict(np.vstack([xpts]*2))
        
r2_ps = []
for i in range(0, 10):
    vn = vnames[i]
    if i == 0:
        r2_ps.append(figure(x_range=r1_ps[i].x_range, **pargs))
    else:
        r2_ps.append(figure(y_range=r2_ps[0].y_range, x_range=r1_ps[i].x_range, **pargs))
    r2_ps[-1].yaxis.visible = False
    r2_ps[-1].xaxis.visible = False
    r2_ps[-1].min_border = 0
    r2_ps[-1].line(x=vn + '_x', y=vn + '_y2', line_width=3, alpha=0.5, line_cap='round', name='r_surf', source=source)
    r2_ps[-1].line(vn+'_bnds', 'r2val', color='black', line_width=1, alpha=0.7, name='r_value', source=psource)
    #r2_ps[-1].toolbar.active_drag = r2_ps[-1].tools[1]
    r2_ps[-1].toolbar.active_scroll = r2_ps[-1].tools[0]
    hover = HoverTool(tooltips=[(vn, "$x"), ("r2", "@{}_y2".format(vn))], mode='vline', names=['r_surf'])
    r2_ps[-1].add_tools(hover)
    r2_ps[-1].grid.grid_line_alpha = 0.4
    callback = CustomJS(args=dict(myText=textrow[i]),
                        code="""myText.value = cb_obj['x'].toPrecision(5);""")
    r2_ps[-1].js_on_event(events.Tap, callback)
    r2_ps[-1].js_on_event(events.Pan, callback)


show(row([column(ti, inp, r1, r2, width=130) for ti, inp, r1, r2 in zip(t0_ps, textrow, r1_ps, r2_ps)]))

KeyError: 'avg_wh_p'

In [15]:
x = np.linspace(scalers[0].data_min_[0], scalers[0].data_max_[0], 32)
y = np.linspace(scalers[1].data_min_[0], scalers[1].data_max_[0], 32)
xx, yy = np.meshgrid(x, y)
w = np.linspace(0, 1, 32)
wx, wy = np.meshgrid(w, w)

In [33]:
wx = wx.ravel()
wy = wy.ravel()

In [18]:
xx = xx.ravel()
yy = yy.ravel()

In [37]:
surfin = np.vstack([wx, wy] + [np.ones(wx.shape) * xm for xm in xpts[2:]])
surfin.shape

(2, 1024)

In [39]:
rr2 = respo2.predict(np.vstack([wx, wy] + [np.ones(wx.shape) * xm for xm in xpts[0][2:]]).T)

In [16]:
output_file('simple_surface.html', title='profile_surface')

surfsource = ColumnDataSource(data=dict(x=xx, y=yy, z=rr1, color=rr1))

surface = Surface3d(x='x', y='y', z='z', color='color')

show(surface)

In [53]:
np.vstack((np.array([[.34, .55], [.65, .88]]).reshape(1, -1), np.array([[.34, .55], [.65, .88]]).reshape(1, -1)))

array([[ 0.34,  0.55,  0.65,  0.88],
       [ 0.34,  0.55,  0.65,  0.88]])

In [19]:
x0 = np.linspace(0, 10, 32)
y0 = np.linspace(50, 500, 32)
xx0, yy0 = np.meshgrid(x, y)
xx0 = xx0.ravel()
yy0 = yy0.ravel()

array([ 700.])

In [17]:
pd.DataFrame({'x':xx, 'y':yy, 'r1':rr1}).to_csv('C:\\users\\jmccrary\\simple_surface\\test_surface_data.csv', index=False)

In [19]:
z0 = np.linspace(0, 1, 32)
z1 = np.linspace(0, 1, 32)
zz0, zz1 = np.meshgrid(z0, z1)
zz0 = zz0.ravel()
zz1 = zz1.ravel()

In [20]:
pd.DataFrame({'x':zz0, 'y':zz1, 'r1':rr1}).to_csv('C:\\users\\jmccrary\\simple_surface\\test_norm_surface_data.csv', index=False)

In [189]:
testcode = """var myName = %s;
var pdata = psource.data;
var g = cb_obj['x'];
var coef1 = %s;
var coef0 = %s;
var cept1 = %s;
var cept0 = %s;
var vnames = %s;
var xzpt = [];
var newrval = 0;
var partval = 0;
var bnds = pdata[myName + '_bnds']
var h = (g - bnds[0]) / (bnds[1] - bnds[0])

var line = "<span style=float:left;clear:left;font_size=0.5pt><b>CALC: " + myName + " " + bnds + "</b> x = " + g + ", z = " + h + ", <b>coef0.shape</b> = (" + coef0.length + ", " + coef0[0].length + ") <b>coef1.shape</b> = (" + coef1.length + ", " + coef1[0].length + ") <b>cept0.shape</b> = (" + cept0.length + ", ) <b>cept.shape</b> = (" + cept1.length + ", ) vnames = " + vnames + "</span>\\n";
var text = outdiv.text.concat(line);
var lines = text.split("\\n")
if ( lines.length > 350 ) { lines.shift(); }
outdiv.text = lines.join("\\n");

pdata[myName + '_p'] = [h, h];

//scale the current point to minmax interval of [0, 1] for proper input into activations
for (i = 0; i < vnames.length; i++) {
  xzpt[i] = pdata[vnames[i] + '_p'][0] + 0
};

var line = "<span style=float:left;clear:left;font_size=0.5pt><b> xzpt:  " + xzpt + "</b></span>\\n";
var text = outdiv.text.concat(line);
var lines = text.split("\\n")
if ( lines.length > 350 ) { lines.shift(); }
outdiv.text = lines.join("\\n");

newrval = 0 + cept1[0]
line = "<span style=float:left;clear:left;font_size=0.5pt><b>Initial newrval = " + newrval + "</b></span>\\n";
text = outdiv.text.concat(line);
lines = text.split("\\n")
if ( lines.length > 350 ) { lines.shift(); }
outdiv.text = lines.join("\\n");

for (j=0; j < coef0[0].length; j++) {
  partval = 0 + cept0[j];
  line = "<span style=float:left;clear:left;font_size=0.5pt><b>__  j = </b>" + j + ", intial partval = " + partval + "</b></span>\\n";
  text = outdiv.text.concat(line);
  lines = text.split("\\n")
  if ( lines.length > 350 ) { lines.shift(); }
  outdiv.text = lines.join("\\n");
  
  for (i=0; i < coef0.length; i++) {
    partval += coef0[i][j] * xzpt[i];
  };
  newrval += Math.tanh(partval) * coef1[j][0];
  line = "<span style=float:left;clear:left;font_size=0.5pt><b>_ j = </b>" + j + ",   <b>newrval</b> = " + newrval + "</b></span>\\n";
  text = outdiv.text.concat(line);
  lines = text.split("\\n")
  if ( lines.length > 350 ) { lines.shift(); }
  outdiv.text = lines.join("\\n");
};
pdata['r1val'] = [newrval, newrval];

psource.change.emit();
""" % ("'%s'", respo1.coefs_[1].tolist(), respo1.coefs_[0].tolist(), respo1.intercepts_[1].tolist(), respo1.intercepts_[0].tolist(), vnames)

In [140]:
showweightsinfo="""
var myName = %s;
var coef0 = %s;
var coef1 = %s;
var cept0 = %s;
var cept1 = %s;
var line = "<span style=float:left;clear:left;font_size=0.5pt><b>" + myName + ": coef0.shape</b> = (" + coef0.length + ", " + coef0[0].length + ") <b>coef1.shape</b> = (" + coef1.length + ", " + coef1[0].length + ") <b>cept0.shape</b> = (" + cept0.length + ", ) <b>cept.shape</b> = (" + cept1.length + ", )</span>\\n";
var text = outdiv.text.concat(line);
var lines = text.split("\\n");
if ( lines.length > 35 ) { lines.shift(); }
outdiv.text = lines.join("\\n");
""" % ("'%s'", respo1.coefs_[0].tolist(), respo1.coefs_[1].tolist(), respo1.intercepts_[0].tolist(), respo1.intercepts_[1].tolist())

In [17]:
r1code = """var myName = %s;
var data = source.data;
var pdata = psource.data;
var g = cb_obj['x'];
var coef1 = %s;
var coef0 = %s;
var cept1 = %s;
var cept0 = %s;
var vnames = %s;

var aixs = new Array(vnames.length);
for (i = 0; i < vnames.length; i++) {
  aixs[i] = i;
};
var myix = vnames.indexOf(myName);
var oixs = aixs.slice();
oixs.splice(myix, 1);
var mixs = [];
var xzpt = [];
var newrval = 0;
var partval = 0;
var bnds = pdata[myName + '_bnds'].slice();
var newy = [];
var h = (g - bnds[0]) / (bnds[1] - bnds[0]);
var xx = data['x_'].slice();
var partsums = []; 

pdata[myName + '_p'] = [h, h];

// scale the current point to minmax interval of [0, 1] for proper input into activations
for (i = 0; i < vnames.length; i++) {
  xzpt[i] = pdata[vnames[i] + '_p'][0] + 0;
};


newrval = 0 + cept1[0];
for (j=0; j < coef0[0].length; j++) {
  partval = 0 + cept0[j];
  
  for (i=0; i < coef0.length; i++) {
    partval += coef0[i][j] * xzpt[i];
  };
  newrval += Math.tanh(partval) * coef1[j][0];
  
};
pdata['r1val'] = [newrval, newrval];

// update other features since current feature line won't change
for (o = 0; o < oixs.length; o++) {
  newy = new Array(xx.length); // initialize array for new curve y values
  for (k = 0; k < xx.length; k++) {
    newy[k] = 0 + cept1[0]; // start with the output activation's intercept
  };
  mixs = aixs.slice(); // copy of indices
  mixs.splice(oixs[o], 1); // drop current feature's index
  
  // make a list of partial weighted sums of the mix features for input to
  // each perceptron so that we don't have to do these operations k times
  partsums = new Array(cept1.length);
  for (j = 0; j < cept0.length; j++) {
    partsums[j] = cept0[j] + 0; // start with each input activation's intercept
    // just add weighted static values to the partsums
    for (i = 0; i < mixs.length; i++) {
      partsums[j] += coef0[mixs[i]][j] * xzpt[mixs[i]];
    };
  };
  
  for (k = 0; k < xx.length; k++) {
    for (j = 0; j < coef0[oixs[o]].length; j++) {
      // add the weighted value of this feature's point
      newy[k] += Math.tanh(partsums[j] + coef0[oixs[o]][j] * xx[k]) * coef1[j][0];
    };
  };
  data[vnames[oixs[o]] + '_y1'] = newy.slice();
};

source.change.emit();
psource.change.emit();
""" % ("'%s'", respo1.coefs_[1].tolist(), respo1.coefs_[0].tolist(), respo1.intercepts_[1].tolist(), respo1.intercepts_[0].tolist(), vnames)

In [10]:
53/783

0.06768837803320563

In [20]:
somedata = pd.read_csv("C://Users//jmccrary//javascript//d3stuff//somedata.csv")


In [21]:
somedata

,name,age,weight,money
0,josh,32.0,200,80800
1,kyoung,35.0,127,23000
2,minho,1.5,24,6700
3,storm,6.0,35,50
4,charlotte,64.0,167,120000
5,john,65.0,208,70000


In [33]:
with open('C://Users//jmccrary//javascript//d3stuff//py_written_somedata.js', 'w') as fio:
    fio.write("var data = " + somedata.to_json(orient='split') + ";")

In [17]:
df.columns

Index(['boost_reliability', 'cep', 'hob', 'launch_reliability', 'missile_type',
       'missiles_expended', 'multi_missile', 'probability_penetrate',
       'rb_reliability', 'single_missile', 'soa', 'avg_warhead_load',
       'var_warhead_load', 'weighted_avg_yld', 'total_wh_expended',
       'total_cost', 'coverage_norm', 'msn_success_norm',
       'Predicted missiles_expended', 'Predicted total_wh_expended',
       'Total_utility', 'Pareto_Utility', 'Predicted Total_utility',
       'Predicted total_cost', 'Predicted coverage_norm',
       'Predicted msn_success_norm'],
      dtype='object')

In [27]:
df[['Total_utility', 'total_cost', 'missile_type', 'total_wh_expended', 'missiles_expended']].to_json(path_or_buf='C://Users//jmccrary//javascript//d3stuff//pareto_data.js', orient='records', lines=True)

In [35]:
df['concept'] = df.missile_type.apply(lambda x: int(x[-1]))

In [36]:
df.concept.unique()

array([3, 1, 2], dtype=int64)

In [37]:
with open('C://Users//jmccrary//javascript//d3stuff//pareto_data.js', 'w', encoding='utf-8') as fio:
    fio.write("var df = " + df[['Total_utility', 'total_cost', 'concept', 'total_wh_expended', 'missiles_expended']].to_json(orient='split') + ";")

In [38]:
from bokeh.core.properties import Any, Dict, Instance, String
from bokeh.models import LayoutDOM

In [42]:
DEFAULTS = {
    'width':          '600px',
    'height':         '600px',
    'style':          'surface',
    'showPerspective': True,
    'showGrid':        True,
    'keepAspectRatio': True,
    'verticalRatio':   1.0,
    'legendLabel':     'stuff',
    'cameraPosition':  {
        'horizontal': -0.35,
        'vertical':    0.22,
        'distance':    1.8,
    }
}

class Surface3d(LayoutDOM):
    __implementation__ = "surface3d.js"
    
    data_source = Instance(ColumnDataSource)
    
    x = String
    
    y = String
    
    z = String
    
    color = String
    
    options = Dict(String, Any, default=DEFAULTS)

In [40]:
from bokeh.models import Div, Column

In [41]:
x = np.arange(0, 300, 100)
y = np.arange(0, 300, 100)
xx, yy = np.meshgrid(x,y)
xx = xx.ravel()
yy = yy.ravel()

def comput(t):
    value = np.sin(xx/50 + t/10) * np.cos(yy/50 + t/10) * 50 + 50
    return dict(x=xx, y=yy, z=value, color=value)

source = ColumnDataSource(data=compute(0))



NameError: name 'x' is not defined

In [3]:
profile = row([column(ti, inp, r1, r2, width=130) for ti, inp, r1, r2 in zip(t0_ps, textrow, r1_ps, r2_ps)])
surftab = Tabs(tabs=[tab1, tab2])

show(column(profile, surftab))

In [25]:
help(tickers.BasicTicker)

Help on class BasicTicker in module bokeh.models.tickers:

class BasicTicker(AdaptiveTicker)
 |  Generate ticks on a linear scale.
 |  
 |  .. note::
 |      This class may be renamed to ``LinearTicker`` in the future.
 |  
 |  Method resolution order:
 |      BasicTicker
 |      AdaptiveTicker
 |      ContinuousTicker
 |      Ticker
 |      bokeh.model.Model
 |      bokeh.core.has_props.HasProps
 |      bokeh.util.callback_manager.PropertyCallbackManager
 |      bokeh.util.callback_manager.EventCallbackManager
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  __cached_all__properties__ = {'base', 'desired_num_ticks', 'js_event_c...
 |  
 |  __container_props__ = set()
 |  
 |  __properties__ = set()
 |  
 |  __properties_with_refs__ = set()
 |  
 |  __view_model__ = 'BasicTicker'
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from AdaptiveTicker:
 |  
 |  base
 |      The multiplier to use 

{'avg_wh_x': array([ 5.15 ,  5.217,  5.283,  5.35 ,  5.417,  5.483,  5.55 ,  5.617,
         5.683,  5.75 ,  5.817,  5.883,  5.95 ,  6.017,  6.083,  6.15 ,
         6.217,  6.283,  6.35 ,  6.417,  6.483,  6.55 ,  6.617,  6.683,
         6.75 ,  6.817,  6.883,  6.95 ,  7.017,  7.083,  7.15 ,  7.217,
         7.283,  7.35 ,  7.417,  7.483,  7.55 ,  7.617,  7.683,  7.75 ,
         7.817,  7.883,  7.95 ,  8.017,  8.083,  8.15 ,  8.217,  8.283,
         8.35 ,  8.417,  8.483,  8.55 ,  8.617,  8.683,  8.75 ,  8.817,
         8.883,  8.95 ,  9.017,  9.083,  9.15 ,  9.217,  9.283,  9.35 ]),
 'avg_wh_y1': array([  1.001e+01,   9.730e+00,   9.438e+00,   9.141e+00,   8.838e+00,
          8.531e+00,   8.219e+00,   7.904e+00,   7.588e+00,   7.270e+00,
          6.952e+00,   6.635e+00,   6.320e+00,   6.007e+00,   5.699e+00,
          5.395e+00,   5.097e+00,   4.806e+00,   4.523e+00,   4.248e+00,
          3.983e+00,   3.727e+00,   3.482e+00,   3.247e+00,   3.023e+00,
          2.811e+00,   2.610e+00